In [ ]:
import numpy as np
import pysindy as ps
from pysindy.feature_library import PolynomialLibrary, FourierLibrary
import matplotlib.pyplot as plt

from generate_data import generate_population_data, generate_discrete_population_data

In [ ]:
T = 1e-1

t = np.linspace(0, 100, 1001)

x0 = [50, 50]
x0_val = [100, 30]

In [ ]:
xk = generate_discrete_population_data(t, x0, T=T)
xk_val = generate_discrete_population_data(t, x0_val, T=T)

In [ ]:
x, x_dot = generate_population_data(t, x0)
x_val, _ = generate_population_data(t, x0_val)

In [ ]:
pr = int(3 - np.log10(T))
model = ps.SINDy(
    feature_library=PolynomialLibrary(degree=2),
    # feature_library=FourierLibrary(n_frequencies=2),
    optimizer=ps.STLSQ(threshold=0.0001),
    feature_names=[f'x{i+1}' for i in range(len(x[0]))],
    discrete_time=True
    )
model.fit(x=xk)
# model.fit(x=x, x_dot=x_dot, t=t)
model.print(precision=pr)

In [ ]:
x_sim = model.simulate(x0=x0_val, t=len(t))
mse_x1 = ((x_sim[:, 0] - xk_val[:, 0])**2).mean()
mse_x2 = ((x_sim[:, 1] - xk_val[:, 1])**2).mean()
print(f'Błąd średniokwadratowy x1: {mse_x1:.2f}, x2: {mse_x2:.2f}')

In [ ]:
plt.plot(t, xk_val[:, 0])
plt.plot(t, xk_val[:, 1])
plt.plot(t, x_sim[:, 0], "r--")
plt.plot(t, x_sim[:, 1], "k--")
plt.legend(["Populacja ofiar", "Populacja drapieżników", "Model ofiar", "Model drapieżników"])
plt.ylim(0, max(xk_val[:, 0]*1.4))
plt.xlim(0, max(t))
plt.xlabel("Czas [dni]")
plt.ylabel("Liczba osobników")
plt.grid()
plt.show()